In [ ]:
# run_gym_qt_route_a.py
import os
os.environ['QT_QPA_PLATFORM'] = 'offscreen'

import json
import sys
import threading
import traceback

# Auto-detect the Food-Delivery-Bench repo root.
from pathlib import Path

cwd = Path.cwd().resolve()
base_dir = None
for p in [cwd, *cwd.parents]:
    # 1) Current directory is the repo root.
    if (p / "vlm_delivery").is_dir() and (p / "simworld").is_dir():
        base_dir = p
        break
    # 2) Repo root is a direct child of the current path.
    candidate = p / "Food-Delivery-Bench"
    if (candidate / "vlm_delivery").is_dir() and (candidate / "simworld").is_dir():
        base_dir = candidate
        break

if base_dir is None:
    raise RuntimeError("Cannot auto-detect Food-Delivery-Bench root.")

base_dir = str(base_dir)
sys.path.insert(0, base_dir)
sys.path.insert(0, os.path.join(base_dir, "simworld"))

from PyQt5.QtCore import QTimer
from vlm_delivery.gym_like_interface.gym_like_interface import DeliveryBenchGymEnvQtRouteA


def main():
    exp_cfg_path = os.path.join(base_dir, "vlm_delivery", "input", "experiment_config.json")
    with open(exp_cfg_path, "r", encoding="utf-8") as f:
        exp_cfg = json.load(f) or {}
    gym_env_cfg = exp_cfg.get("gym_env", {}) or {}

    env = DeliveryBenchGymEnvQtRouteA(
        base_dir=base_dir,
        ue_ip=gym_env_cfg.get("ue_ip", "127.0.0.1"),
        ue_port=int(gym_env_cfg.get("ue_port", 9015)),
        sim_tick_ms=100,
        vlm_pump_ms=100,
        enable_viewer=True,  # If unstable, try False to isolate viewer issues.
        map_name=gym_env_cfg.get("map_name", "medium-city-22"),
        max_steps=20,
    )

    # 1) Must run on the main thread: create QApplication + invoker.
    env.bootstrap_qt()

    def rl_loop():
        try:
            # 2) Run reset/step on a worker thread.
            obs, info = env.reset(seed=0)
            print("reset info:", info)
            print("obs:", obs)

            for step_i in range(1, 999999):
                obs, r, term, trunc, info2 = env.step(None)
                print(f"[RL] step={step_i} info:", info2)

                # Print any error with tracebacks.
                if info2.get("error"):
                    print("STEP ERROR:", info2["error"])
                    if info2.get("dispatch_exc"):
                        print("DISPATCH TRACEBACK:\n", info2["dispatch_exc"])
                    if info2.get("enqueue_exc"):
                        print("ENQUEUE TRACEBACK:\n", info2["enqueue_exc"])
                    break

                if term or trunc:
                    break

        except Exception as e:
            print("[RL] Exception:", e)
            traceback.print_exc()

        finally:
            try:
                env.close()
            except Exception:
                pass
            try:
                if getattr(env, "_app", None) is not None:
                    env._app.quit()
            except Exception:
                pass

    # 3) Start the RL thread after the Qt loop is ready.
    QTimer.singleShot(0, lambda: threading.Thread(target=rl_loop, daemon=True).start())

    # 4) Run the Qt event loop on the main thread.
    env.run_qt_loop()


if __name__ == "__main__":
    main()


/home/lingjun/miniconda3/envs/simworld/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/lingjun/miniconda3/envs/simworld/lib/python3.10/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
INFO:__init__:230:Got connection confirm: b'connected to gym_citynav'


Loaded bus route: route_bus_1 with 8 stops
Created bus bus_1 on route route_bus_1
=>Info: using ip-port socket


This plugin does not support propagateSizeHints()


reset info: {'sim_time': None, 'seed': 0, 'options': {}, 'sim_tick_ms': 100, 'vlm_pump_ms': 100, 'run_dir': '/home/lingjun/embodied-agent-framework/DeliveryBench-release/outputs/trajectories/run_20260131_005615'}
obs: {'state': array([ -8.115, -12.354,   0.   ,   0.   ,   0.   ], dtype=float32)}
[GRAB] py= MainThread qt_id= 140407724197696 has_app= True is_gui= True
invoker elapsed: 6.0973145961761475 ok= True exc= None


2026-01-31 00:56:28 - delivery_system.agent_DeliveryMan1 - INFO - [Agent 1] [VLM] raw output: {
"reasoning_and_reflection": "Scooter at origin with 50% battery and full energy. No active orders; to maximize earnings I should immediately look for available orders rather than moving or charging.",
"action": "VIEW_ORDERS()",
"future_plan": "Accept the best nearby multi-stop order(s), ride to the pickup, manage temperature with packs if needed, arrange items in bag, then navigate efficiently to drop-offs. Monitor energy and scooter battery; charge only if necessary between runs."
}
2026-01-31 00:56:28 - delivery_system.agent_DeliveryMan1 - INFO - [VLM] parsed action: view_orders 
2026-01-31 00:56:28 - delivery_system.agent_DeliveryMan1 - INFO - [Agent 1] view orders


...

Starting action: DMAction(kind=<DMActionKind.PLACE_FOOD_IN_BAG: 'place_food_in_bag'>, data={'bag_cmd': 'order 6: 1 -> A'}, on_done=None)
DMAction(kind=<DMActionKind.PLACE_FOOD_IN_BAG: 'place_food_in_bag'>, data={'bag_cmd': 'order 6: 1 -> A'}, on_done=None)
[RL] step=6 info: {'sim_time': None, 'elapsed_steps': 6, 'reward_info': {'base': 0.0}, 'termination_info': {}, 'mode': 'vlm_decision', 'done0': 5, 'done1': 6}
[GRAB] py= MainThread qt_id= 140407724197696 has_app= True is_gui= True
invoker elapsed: 5.171876907348633 ok= True exc= None


2026-01-31 00:58:48 - delivery_system.agent_DeliveryMan1 - INFO - [Agent 1] [VLM] raw output: {
"reasoning_and_reflection": "Order 6 is already in bag and due in 1 min; closest path is east to S1 then south to 5th road left to the dropoff at (-176.97,-125.79). Battery 38% is enough; energy fine. Prioritize quick delivery of #6, then return for #5 when ready.",
"action": "MOVE(-176.97m, -125.79m)",
"future_plan": "Upon arrival, DROP_OFF(oid=6, method=\"leave_at_door\"). Then ride back to restaurant at (-288.53m,-16.58m) to PICKUP order 5 when ready, bag it, and deliver to (-304.10m,-224.68m). Monitor battery; charge if below 20% after deliveries."
}
2026-01-31 00:58:48 - delivery_system.agent_DeliveryMan1 - INFO - [VLM] parsed action: move_to {'tx': -17697.0, 'ty': -12579.0, 'use_route': True, 'snap_cm': 120.0}
2026-01-31 00:58:48 - delivery_system.agent_DeliveryMan1 - INFO - [Agent 1] move from (-287.18m, -16.71m) to (-176.97m, -125.79m) [mode=e-scooter, speed=600.0 cm/s, pace=normal]


Starting action: DMAction(kind=<DMActionKind.MOVE_TO: 'move_to'>, data={'tx': -17697.0, 'ty': -12579.0, 'use_route': True, 'snap_cm': 120.0}, on_done=None)
DMAction(kind=<DMActionKind.MOVE_TO: 'move_to'>, data={'tx': -17697.0, 'ty': -12579.0, 'use_route': True, 'snap_cm': 120.0}, on_done=None)
[RL] step=7 info: {'sim_time': None, 'elapsed_steps': 7, 'reward_info': {'base': 0.0}, 'termination_info': {}, 'mode': 'vlm_decision', 'done0': 6, 'done1': 7}
[GRAB] py= MainThread qt_id= 140407724197696 has_app= True is_gui= True
invoker elapsed: 4.942969560623169 ok= True exc= None


2026-01-31 00:59:15 - delivery_system.agent_DeliveryMan1 - INFO - [Agent 1] [VLM] raw output: {
"reasoning_and_reflection": "I arrived at order #6’s drop-off address with the food already bagged. Battery at 30% is sufficient for the short next leg. Order #5 is still being prepared but nearly ready. I should complete #6 immediately to avoid overtime penalties.",
"action": "DROP_OFF(oid=6, method=\"leave_at_door\")",
"future_plan": "Scoot back to the restaurant at (-288.53m,-16.58m) to pick up order #5 as soon as it’s ready, bag it, then head to its drop-off at (-304.10m,-224.68m). If scooter battery drops near 20%, stop by the nearby charger on 11th road to top up."}
2026-01-31 00:59:15 - delivery_system.agent_DeliveryMan1 - INFO - [VLM] parsed action: drop_off {'oid': 6, 'method': 'leave_at_door'}
2026-01-31 00:59:15 - delivery_system.agent_DeliveryMan1 - INFO - [Agent 1] dropped off order #6 (extra +1.50, stars=4) [time=4, food=4, method=5] [on_time=Y, temp=BAD, odor=OK, damage=OK]


Starting action: DMAction(kind=<DMActionKind.DROP_OFF: 'drop_off'>, data={'oid': 6, 'method': 'leave_at_door'}, on_done=None)
DMAction(kind=<DMActionKind.DROP_OFF: 'drop_off'>, data={'oid': 6, 'method': 'leave_at_door'}, on_done=None)
[RL] step=8 info: {'sim_time': None, 'elapsed_steps': 8, 'reward_info': {'base': 0.0}, 'termination_info': {}, 'mode': 'vlm_decision', 'done0': 7, 'done1': 8}
